In [1]:
# Import Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [2]:
# Load the data
data = pd.read_csv('Data/occupation_skills.csv')

In [3]:
""" Built an application that will take an occupation as input and return the top 10 skills required for that occupation. 
The application should also return the top 10 occupations that require the same skill. 
The application should be able to handle cases where the occupation or skill is not in the dataset.
THe application should be able to lookup the occupation and skill by their labels and not  by their URIs.
The application should be able to handle cases where the occupation or skill label is not an exact match to the label in the dataset,
but matches the altLabel in the dataset.
"""
def find_top_skills_for_occupation(occupation_input, data):
    # Normalize input and data for case-insensitive matching
    occupation_input_normalized = occupation_input.strip().lower()
    
    # Ensure the alt_occupationLabel_normalized column is a list even if originally NaN
    # This adjustment is done inline during the operation that caused the error
    data['alt_occupationLabel_normalized'] = data['alt_occupationLabel'].str.lower().str.split('\n').apply(lambda x: x if isinstance(x, list) else [])
    
    # Attempt to match the input with both the occupationLabel and alt_occupationLabel
    # Check direct matches in occupationLabel
    direct_matches = data[data['occupationLabel'].str.lower() == occupation_input_normalized]
    
    # If no direct matches, check for matches in alt_occupationLabel
    if direct_matches.empty:
        matches = data[data['alt_occupationLabel_normalized'].apply(lambda x: occupation_input_normalized in x if isinstance(x, list) else False)]
    else:
        matches = direct_matches
    
    # If no matches found, run the find_top_skills_for_occupation_partial_matches function
    if matches.empty:
        return find_top_skills_for_occupation_partial_matches(occupation_input, data)
    
    # Aggregate skills and their frequencies
    skills_frequency = matches['skillLabel'].value_counts().head(10)
    
    # Return the top 10 skills for the occupation
    return skills_frequency


def find_top_skills_for_occupation_partial_matches(occupation_input, dataset, min_length=2):
    # Local preprocessing within the function
    temp_data = dataset.copy()
    temp_data['alt_occupationLabel'] = temp_data['alt_occupationLabel'].str.lower().str.split('\n').apply(lambda x: x if isinstance(x, list) else [])
    
    occupation_input_normalized = occupation_input.lower().strip()

    # Check for partial matches without altering the original dataset structure
    def matches_occupation(row):
        if occupation_input_normalized in row['occupationLabel'].lower():
            return True
        return any(occupation_input_normalized in alt for alt in row['alt_occupationLabel'])

    filtered_data = temp_data[temp_data.apply(matches_occupation, axis=1)]

    # If no matches are found and the occupation_input is longer than min_length, shorten it
    if filtered_data.empty:
        if len(occupation_input_normalized) > min_length:
            # Shorten the occupation_input by removing the last character or last word
            # Here, we'll remove the last word for better chances of meaningful matches
            shortened_input = ' '.join(occupation_input_normalized.split()[:-1])
            if shortened_input:
                return find_top_skills_for_occupation_partial_matches(shortened_input, dataset, min_length)
            else:
                # Call find_occupations_for_skill here with the original input before returning no matches
                occupations_result = find_occupations_for_skill(occupation_input, dataset)
                if occupations_result == f"No matches found for skill: {occupation_input}":
                    return "Match not found"
                return occupations_result
        else:
            # Call find_occupations_for_skill here with the original input before returning no matches
            occupations_result = find_occupations_for_skill(occupation_input, dataset)
    
    skills_frequency = filtered_data['skillLabel'].value_counts().head(10)
    return skills_frequency

def find_occupations_for_skill(skill_input, data):
    # Copy the data to avoid modifying the original dataframe
    temp_data = data.copy()
    
    # Normalize and preprocess for exact and partial matching
    skill_input_normalized = skill_input.lower().strip()
    temp_data['skillLabel_normalized'] = temp_data['skillLabel'].str.lower()
    temp_data['alt_skillLabel_normalized'] = temp_data['alt_skillLabel'].apply(
        lambda x: x.lower().split('\n') if isinstance(x, str) else []
    )
    
    # First, attempt to find exact matches
    direct_matches = temp_data[temp_data['skillLabel_normalized'] == skill_input_normalized]
    
    if direct_matches.empty:
        # If no exact matches, attempt to find matches in alternative labels
        matches = temp_data[temp_data['alt_skillLabel_normalized'].apply(
            lambda x: skill_input_normalized in x)]
    else:
        matches = direct_matches
    
    # If no matches found yet, look for partial matches within skill labels and their alternatives
    if matches.empty:
        matches = temp_data[
            temp_data['skillLabel_normalized'].str.contains(skill_input_normalized) |
            temp_data['alt_skillLabel_normalized'].apply(lambda x: any(skill_input_normalized in alt for alt in x))
        ]
    
    if matches.empty:
        return f"No matches found for: {skill_input}"
    
    # Aggregate the occupations and count their frequencies
    occupations_frequency = matches['occupationLabel'].value_counts().head(10)
    
    # Return the top 10 occupations requiring the skill
    return occupations_frequency


In [4]:
# Testing Functions for welder
test_occupation = "welder"

find_top_skills_for_occupation(test_occupation, data)

skillLabel
cutting technologies                         1
manufacturing of steam generators            1
perform welding inspection                   1
apply preliminary treatment to workpieces    1
determine suitability of materials           1
assemble metal parts                         1
prepare pieces for joining                   1
apply soldering techniques                   1
operate brazing equipment                    1
operate automated process control            1
Name: count, dtype: int64

In [5]:
# Testing Functions for partial welder
test_occupation_partial = "qa"

find_top_skills_for_occupation_partial_matches(test_occupation_partial, data)


skillLabel
inspect quality of products              6
maintain test equipment                  6
use measurement instruments              5
quality assurance procedures             4
write inspection reports                 4
quality standards                        4
record test data                         4
use testing equipment                    4
operate precision measuring equipment    4
oversee quality control                  4
Name: count, dtype: int64

In [6]:
# Testing for "Bolt Torquing Technical"
test_skill = "Bolt Torquing Technical"
find_top_skills_for_occupation(test_skill, data)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# def find_occupations_for_skill(skill_input, data):
#     # Copy the data to avoid modifying the original dataframe
#     temp_data = data.copy()
    
#     # Normalize and preprocess for exact and partial matching
#     skill_input_normalized = skill_input.lower().strip()
#     temp_data['skillLabel_normalized'] = temp_data['skillLabel'].str.lower()
#     temp_data['alt_skillLabel_normalized'] = temp_data['alt_skillLabel'].apply(
#         lambda x: x.lower().split('\n') if isinstance(x, str) else []
#     )
    
#     # First, attempt to find exact matches
#     direct_matches = temp_data[temp_data['skillLabel_normalized'] == skill_input_normalized]
    
#     if direct_matches.empty:
#         # If no exact matches, attempt to find matches in alternative labels
#         matches = temp_data[temp_data['alt_skillLabel_normalized'].apply(
#             lambda x: skill_input_normalized in x)]
#     else:
#         matches = direct_matches
    
#     # If no matches found yet, look for partial matches within skill labels and their alternatives
#     if matches.empty:
#         matches = temp_data[
#             temp_data['skillLabel_normalized'].str.contains(skill_input_normalized) |
#             temp_data['alt_skillLabel_normalized'].apply(lambda x: any(skill_input_normalized in alt for alt in x))
#         ]
    
#     if matches.empty:
#         return f"No matches found for skill: {skill_input}"
    
#     # Aggregate the occupations and count their frequencies
#     occupations_frequency = matches['occupationLabel'].value_counts().head(10)
    
#     # Return the top 10 occupations requiring the skill
#     return occupations_frequency

In [ ]:
# Testing function for skill "welding"
test_skill = "Bolt Torquing Technical"

find_occupations_for_skill(test_skill, data)


In [ ]:
# Load test data
test_data = pd.read_csv('Test_occupations.csv')
test_data.head()

In [ ]:
# Search for top skills for each job_title in the test data one by one and print results
for occupation in test_data['Occupations']:
    test_title = occupation
    print(f"Top skills for {test_title}:")
    print(find_top_skills_for_occupation(test_title, data))
    print("\n")
    